# What to do

Over-/Under-sampling yielded better result than without these techniques. Still not satisfying. More reasonable filtering needed.
This should be done based on the #tweets-per-day, a maximum time interval between activity and a given overall active time period, eg. 1 tpd, max 5 days between dates, 3 months of tweets.
Try implementing type of hyperparameter tuning for filter conditions.

In [3]:

import pandas as pd

In [4]:
#depression tweets
dt = pd.read_csv("data/depression_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#dt["created_at"] = pd.to_datetime(dt["created_at"])
#dt.set_index('created_at', inplace=True)

#sample tweets
st = pd.read_csv("data/sample_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#st["created_at"] = pd.to_datetime(st["created_at"])
#st.set_index('created_at', inplace=True)

pairs = pd.read_csv('pairs_tpd_users.csv')
print(pairs)

      Unnamed: 0 user id  tweets per day
0           1330  uR1330               1
1           5065  uR5065               1
2           1259  uR1259               1
3           5069  uR5069               1
4           3405  uR3405               1
...          ...     ...             ...
7344         259  uR0259             348
7345        7207  uR7207             356
7346        5920  uR5920             624
7347        6361  uR6361             892
7348          16  uR0016            1060

[7349 rows x 3 columns]


In [25]:
int(pairs[pairs['user id'] == 'uR0000'].iloc[0,2])

2

In [13]:
def get_min_tpd(pairs, users, min_tpd: int):
    """returns list containing all users from data with at average at least min_tpd tweets per day"""
    res = []
    for user in users:
        if int(pairs[pairs['user id'] == user].iloc[0,2]) >= min_tpd:
            res.append(user)
    return res

def get_max_interval_time(data, users, max_days: int):
    """returns list containing all users from data with a maximum of max_days days between tweets"""
    #res = []
    
    start = data.groupby('user_id').min()['created_at']
    end = data.groupby('user_id').max()['created_at']
    span = (end-start).apply(lambda x: x.days)
    u = (span/data.groupby('user_id').count()['created_at']) <= max_days
    return list(u.index[u])
    #for user in users:
    #    tmp = data[data['user_id'] == user]
    #    start = tmp['created_at'].iloc[0]
    #    end = tmp['created_at'].iloc[-1]
    #    span = (start-end).days
    #    if span/len(tmp) <= max_days:
    #        res.append(user)      
    #return res

def get_min_activity(data, users, days: int):
    """returns list dataframe containing all users form data for with data is available for at least days days"""
    start = data.groupby('user_id').min()['created_at']
    end = data.groupby('user_id').max()['created_at']
    span = (end-start).apply(lambda x: x.days)
    u = span > days
    return list(u.index[u])
    #res = []
    #for user in users:
    #    tmp = data[data['user_id'] == user]
    #    start = tmp['created_at'].iloc[0]
    #    end = tmp['created_at'].iloc[-1]
    #    span = (start-end).days
    #    if span > days:
    #        res.append(user)
    #return res

def filter_data(data, users, tpd_pairs, min_tpd: int, max_days_bt: int, min_days_active: int):
    res = []
    for user in users:
        tmp = data[data['user_id'] == user]
        #tmp['created_at'] = pd.to_datetime(tmp['created_at'])
        tmp.loc[:, 'created_at'] = pd.to_datetime(tmp.loc[:, 'created_at'])

        #tweets per day
        if int(tpd_pairs[tpd_pairs['user id'] == user].iloc[0,2]) < min_tpd:
            continue

        #min activity span
        start = tmp['created_at'].iloc[0]
        end = tmp['created_at'].iloc[-1]
        span = (start-end).days
        if span < min_days_active:
            continue

        #max time between tweets
        #counter = 0
        #days_summed = 0
        #for i in range(0, len(tmp.index)-1):
        #    days_summed = days_summed + (tmp['created_at'].iloc[i] - tmp['created_at'].iloc[i+1]).days
        #    counter = counter +1
        #if counter == 0 or days_summed/counter > max_days_bt :
        #    continue
        if span/len(tmp) > max_days_bt:
            continue

        res.append(user)

    return res

filtered_users = []
st.loc[:, 'created_at'] = pd.to_datetime(st.loc[:, 'created_at'])
#min_tpd: int, max_days_bt: int, min_days_active: int
for min_tpd in [1,2,3,4,5,6,7,8,9,10]:
    users_filtered = get_min_tpd(pairs, st.user_id.unique(),min_tpd)
    for max_days_bt in [1,2,3,4,5,6,7]:
        data = st[st['user_id'].isin(set(users_filtered))]
        users_filtered = get_max_interval_time(data, users_filtered, max_days_bt)
        filtered_users.append({'min_tpd': min_tpd, 'max_days_bt': max_days_bt, 'min_days_active': 1, 'user_list': users_filtered})
        print('==== min_tpd = '+str(min_tpd)+', max_days_bt = '+str(max_days_bt)+', min_days_active = '+str(1)+' done ====')
         #for min_days_active in [30,60,90,120,150,180,210,240,270,300,330,360]:
         #    data2 = st[st['user_id'].isin(set(users_filtered))]
         #    users_filtered = get_min_activity(data2, users_filtered, min_days_active)
         #    filtered_users.append({'min_tpd': min_tpd, 'max_days_bt': 1, 'min_days_active': min_days_active, 'user_list': users_filtered})
         #    print('==== min_tpd = '+str(min_tpd)+', max_days_bt = '+str(1)+', min_days_active = '+str(min_days_active)+' done ====')

==== min_tpd = 1, max_days_bt = 1, min_days_active = 1 done ====
==== min_tpd = 1, max_days_bt = 2, min_days_active = 1 done ====
==== min_tpd = 1, max_days_bt = 3, min_days_active = 1 done ====
==== min_tpd = 1, max_days_bt = 4, min_days_active = 1 done ====
==== min_tpd = 1, max_days_bt = 5, min_days_active = 1 done ====
==== min_tpd = 1, max_days_bt = 6, min_days_active = 1 done ====
==== min_tpd = 1, max_days_bt = 7, min_days_active = 1 done ====
==== min_tpd = 2, max_days_bt = 1, min_days_active = 1 done ====
==== min_tpd = 2, max_days_bt = 2, min_days_active = 1 done ====
==== min_tpd = 2, max_days_bt = 3, min_days_active = 1 done ====
==== min_tpd = 2, max_days_bt = 4, min_days_active = 1 done ====
==== min_tpd = 2, max_days_bt = 5, min_days_active = 1 done ====
==== min_tpd = 2, max_days_bt = 6, min_days_active = 1 done ====
==== min_tpd = 2, max_days_bt = 7, min_days_active = 1 done ====
==== min_tpd = 3, max_days_bt = 1, min_days_active = 1 done ====
==== min_tpd = 3, max_day

In [19]:
def get_max_interval_time(data, users, max_days: int):
    """returns list containing all users from data with a maximum of max_days days between tweets"""
    #res = []
    
    start = data.groupby('user_id').min()['created_at']
    end = data.groupby('user_id').max()['created_at']
    span = (end-start).apply(lambda x: x.days)
    u = (span/data.groupby('user_id').count()['created_at']) <= max_days
    return list(u.index[u])

get_max_interval_time(st, [], 23)


['uR0000',
 'uR0001',
 'uR0002',
 'uR0003',
 'uR0004',
 'uR0005',
 'uR0006',
 'uR0007',
 'uR0008',
 'uR0009',
 'uR0010',
 'uR0011',
 'uR0012',
 'uR0014',
 'uR0015',
 'uR0016',
 'uR0017',
 'uR0019',
 'uR0020',
 'uR0021',
 'uR0022',
 'uR0023',
 'uR0024',
 'uR0025',
 'uR0026',
 'uR0027',
 'uR0029',
 'uR0030',
 'uR0031',
 'uR0032',
 'uR0033',
 'uR0034',
 'uR0035',
 'uR0036',
 'uR0037',
 'uR0039',
 'uR0040',
 'uR0041',
 'uR0042',
 'uR0043',
 'uR0044',
 'uR0046',
 'uR0047',
 'uR0048',
 'uR0050',
 'uR0051',
 'uR0052',
 'uR0053',
 'uR0054',
 'uR0056',
 'uR0057',
 'uR0058',
 'uR0059',
 'uR0060',
 'uR0062',
 'uR0063',
 'uR0064',
 'uR0065',
 'uR0066',
 'uR0067',
 'uR0068',
 'uR0069',
 'uR0070',
 'uR0071',
 'uR0072',
 'uR0073',
 'uR0074',
 'uR0075',
 'uR0076',
 'uR0077',
 'uR0078',
 'uR0079',
 'uR0080',
 'uR0081',
 'uR0082',
 'uR0083',
 'uR0084',
 'uR0085',
 'uR0086',
 'uR0088',
 'uR0090',
 'uR0091',
 'uR0092',
 'uR0093',
 'uR0094',
 'uR0096',
 'uR0098',
 'uR0099',
 'uR0100',
 'uR0101',
 'uR0102',

In [24]:
filtered_users = []
st.loc[:, 'created_at'] = pd.to_datetime(st.loc[:, 'created_at'])
#min_tpd: int, max_days_bt: int, min_days_active: int
for min_tpd in [1,2,3,4,5,6,7,8,9,10]:
    users_filtered = get_min_tpd(pairs, st.user_id.unique(),min_tpd)
    for max_days_bt in [1,2,3,4,5,6,7]:
        data = st[st['user_id'].isin(set(users_filtered))]
        users_filtered = get_max_interval_time(data, users_filtered, max_days_bt)
        for min_days_active in [30,60,90,120,150,180,210,240,270,300,330,360]:
            data2 = data[data['user_id'].isin(set(users_filtered))]
            users_filtered = get_min_activity(data2, users_filtered, min_days_active)
            filtered_users.append({'min_tpd': min_tpd, 'max_days_bt': max_days_bt, 'min_days_active': min_days_active, 'user_list': users_filtered})
            print('==== min_tpd = '+str(min_tpd)+', max_days_bt = '+str(max_days_bt)+', min_days_active = '+str(min_days_active)+' done ====')


==== min_tpd = 1, max_days_bt = 1, min_days_active = 30 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 60 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 90 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 120 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 150 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 180 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 210 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 240 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 270 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 300 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 330 done ====
==== min_tpd = 1, max_days_bt = 1, min_days_active = 360 done ====
==== min_tpd = 1, max_days_bt = 2, min_days_active = 30 done ====
==== min_tpd = 1, max_days_bt = 2, min_days_active = 60 done ====
==== min_tpd = 1, max_days_bt = 2, min_days_active = 90 done ====
=

In [14]:
df_filtered = pd.DataFrame(filtered_users)
df_filtered.to_csv('filtered_user_lists_only_tpd_maxtime.csv')

In [16]:
df = pd.DataFrame(filtered_users)
df

,min_tpd,max_days_bt,min_days_active,user_list
65,10,3,1,"[uR0010, uR0011, uR0014, uR0015, uR0016, uR002..."
66,10,4,1,"[uR0010, uR0011, uR0014, uR0015, uR0016, uR002..."
67,10,5,1,"[uR0010, uR0011, uR0014, uR0015, uR0016, uR002..."
68,10,6,1,"[uR0010, uR0011, uR0014, uR0015, uR0016, uR002..."
69,10,7,1,"[uR0010, uR0011, uR0014, uR0015, uR0016, uR002..."
